[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/jrkasprzyk/CVEN5393/blob/main/streamflow_nowak.ipynb)

*This notebook is part of course notes for CVEN 5393: Water Resource Systems and Management, by Prof. Joseph Kasprzyk at CU Boulder.*

This notebook implements the method published below:

Nowak, K., J. Prairie, B. Rajagopalan, and U. Lall (2010), A nonparametric stochastic approach for multisite
disaggregation of annual to daily streamflow, Water Resour. Res., 46, W08529. [DOI](https://doi.org/10.1029/2009WR008530).

In [ ]:
import pandas as pd #for dataframes and data processing
import numpy as np #for numerical computation
import matplotlib.pyplot as plt #for plotting
import sys #system functions
from scipy import interpolate #bring in only the interpolate function
import plotly.express as px #plotly express for fast interactive plotting
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [ ]:
def simulate_flow(rng, Z_input, p_input, years_input, sim_Z):

  #inputs:
  #rng - random number generator instance
  #Z_input - sequence of aggregated flows
  #p_input - 2d array of proportion vectors from historical record
  #years_input - sequence of historical years
  #sim_Z - simulated aggregated flow

  #return: sequence of simulated, disaggregated flow

  # Calculate the distance between the yearly flows and the
  # simulated value
  dist = np.absolute(Z_input - sim_Z)

  # inds will be the indices of the original sequence in ascending order
  inds = dist.argsort()

  # these are the yearly flows, sorted by their distance from the simulated flow
  sorted_Z = paper_Z[inds]

  # the number of neighbors is a function of the
  # number of datapoints in the yearly sequence
  K = int(np.floor(np.sqrt(len(Z_input))))

  # the weight function gives the most weight
  # to the first neighbor (eq 1 in the paper) 
  W = np.zeros(K)
  for i in range(K):
    W[i] = (1./(i+1.))/sum(1./k for k in range(1, K+1))

  # here, we only keep the K nearest neighbors based on distance
  neighbors = sorted_Z[0:K]
  neighbors_inds = inds[0:K]

  # the index of the closest year...
  chosen_index = rng.choice(neighbors_inds, size=1, p=W)

  # ...is used to find a proportion vector
  sim_p = p_input[chosen_index, :]

  # the simulated flow sequence is the proportion multiplied by simulated yearly flow
  sim_flow = sim_p * sim_Z

  print("Sim annual %f, choose year %d, yields: %s" % (sim_Z, years_input[chosen_index], sim_flow))

  return sim_flow

Below, we reproduce the example from the paper!

In [ ]:
my_rng = np.random.default_rng(seed=42)

paper_years = np.array([1967, 1968, 1969, 1970, 1971, 1972, 1973, 1974, 1975])
paper_Z = np.array([35., 40., 33., 52., 43., 56., 38., 49., 32.])
paper_p = np.array(
    [[.1, .3, .4, .2], 
     [.15, .25, .35, .25], 
     [.1, .2, .5, .2],
     [.5, .15, .65, .15],
     [.2, .2, .4, .2],
     [.1, .2, .4, .3],
     [.15, .2, .4, .25],
     [.05, .1, .8, .05],
     [.2, .2, .5, .1]
     ]
      )



Repeating the simulation multiple times shows how different nearest neighbors are selected and thus yield different simulated disaggregated flows

In [ ]:
for i in range(10):
  paper_flows = simulate_flow(my_rng, paper_Z, paper_p, paper_years, 70.0)


Sim annual 70.000000, choose year 1970, yields: [[35.  10.5 45.5 10.5]]
Sim annual 70.000000, choose year 1972, yields: [[ 7. 14. 28. 21.]]
Sim annual 70.000000, choose year 1974, yields: [[ 3.5  7.  56.   3.5]]
Sim annual 70.000000, choose year 1974, yields: [[ 3.5  7.  56.   3.5]]
Sim annual 70.000000, choose year 1970, yields: [[35.  10.5 45.5 10.5]]
Sim annual 70.000000, choose year 1972, yields: [[ 7. 14. 28. 21.]]
Sim annual 70.000000, choose year 1972, yields: [[ 7. 14. 28. 21.]]
Sim annual 70.000000, choose year 1972, yields: [[ 7. 14. 28. 21.]]
Sim annual 70.000000, choose year 1972, yields: [[ 7. 14. 28. 21.]]
Sim annual 70.000000, choose year 1970, yields: [[35.  10.5 45.5 10.5]]
